# Robin's Viewer Demo
## EEG data loading, visualization, and preprocessing with MNE-Python and Robin's Viewer (RV).

<font size="3">Contents:</font> 
* Load demo EEG recording
* Bandpass-filter the signal
* Downsample the signal for viewing
* Clean the filtered signal
    * Mark bad channels in the filtered signal using RV
    * Mark transient artifacts using RV (guided by the deep-learning model)

In [ ]:
import os

import mne

In [ ]:
# Input folder and demo files
data_folder = './data'
EEG_file_name = "RV_demo_signal.raw"
montage_file_name = 'GSN129.sfp'

# Output folder
save_files_folder = './save_files'

# File paths
EEG_file_path = os.path.join(data_folder, EEG_file_name)
montage_file_path = os.path.join(data_folder, montage_file_name)

### 0. Download example EEG recording
<font size="3">In order to run this demo, please download the example data ("RV_demo_signal.raw" and "GSN129.sfp") from https://www.dropbox.com/sh/6llqont8px1s86b/AADmEONSZqmhFXfl5e7NcB8Ga?dl=0 (also linked to in the **"README.md"** file) and store it in the *"data"* directory. The cell below checks whether the files are in the right directory.</font> 

In [ ]:
if not os.path.exists(EEG_file_path):
    print('Example EEG signal not found!')
else:
    print('Found example EEG signal!')
    
if not os.path.exists(montage_file_path):
    print('Montage file not found!')
else:
    print('Found montage file!')

### 1. Load a raw EEG recording

In [ ]:
# Load demo data by using the read_raw_egi() function from MNE.
raw = mne.io.read_raw_egi(EEG_file_path, preload=True, verbose=False)
# Channel E129 is Cz and is the reference channel, let's rename
raw.rename_channels({'E129':'Cz'})
# Drop non-EEG channels
raw.drop_channels(['eECR', 'info', 'sECR', 'STI 014'])

In [ ]:
# Read and set montage
montage = mne.channels.read_custom_montage(montage_file_path)
raw.set_montage(montage)

### 2. Bandpass- and notch-filter the signal
<font size="3">As an example, we will apply a bandpass- (1-45Hz) and notch-filter (50Hz) outside of RV before we load and plot it.</font>

In [ ]:
raw_copy = raw.copy()
bandpass_lfreq = 1
bandpass_hfreq = 45
notch_freq = 50
filtered_signal = raw_copy.filter(bandpass_lfreq, bandpass_hfreq, fir_window='hamming', fir_design="firwin", verbose=0)
filtered_signal.notch_filter(notch_freq, fir_design="firwin", verbose=0)

### 3. Clean the filtered signal
<font size="3">We will demonstrate how to use RV to mark bad channels and transient artifacts.</font>

#### Starting RV 
<font size="3">To start RV, run the cell below. You will see a link where the interactive web application is running. Copy this link into the webbrowser of your choice. The initial screen of RV contains a pop-up window where the EEG data is loaded and all of the preprocessing and visualization settings are defined. This is also where you activate the deep-learning model.</font>

<font size="3">To increase the performance of interactive visualization, the data will be downsampled. This can be done using the parameter in the **"Resampling rate"** field, the value for which is pre-defined in the following cell. To further speed up the performance, we segment the recording into 30-second segments, which can be changed through the **"Segments (in sec)"** field.</font>

<font size="3">To display the EEG signal, press the **"Plot"** button, which will close the pop-up window and lead you to a loading screen after which the main GUI and the signal will be displayed. Note that pressing the **"Plot"** button again will reset your view including all marked bad channels and annotated artifacts.</font>

<font size="3">Use the "->" and "<-" buttons to move to the next or previous 10 seconds, respectively.</font>

##### Marking bad channels in RV

<font size="3">In the menu bar at the top, there is a dropdown menu which is used to select channels to mark them as "bad". When this dropdown menu is clicked, all EEG channels are listed for selection. Additionally, you can type the channels’ names yourself and hence narrow down the selection of options. After having selected bad channels, you will have to redraw the signal (press **"Redraw"** button to the right) in order for these channels to be marked in gray and for the **"Hide/show bad channels"** button to become active. If the loaded recording already includes marked bad channels, they will automatically be selected in this dropdown menu upon loading.</font>

##### Annotate artifacts using RV (guided by a deep-learning model)
<font size="3">As a guidance to detect artifacts, you can use the deep-learning model which was trained to detect artifacts in EEG data (Diachenko, M., Houtman, S. J., Juarez-Martinez, E. L., Ramautar, J. R., Weiler, R., Mansvelder, H. D., Bruining, H., Bloem, P., & Linkenkaer-Hansen K. (2022). Improved manual annotation of EEG signals through convolutional neural network guidance. eNeuro. Unpublished manuscript.). It might help you make decisions when annotating the signal. Tick the box "*run model from run_model.py*" in the pop-up window when RV is started (**Note**: it may take some time to run). Additionally, you can also choose to automatically annotate the signal by the model by ticking the box "*Annotate according to model with threshold*" and typing in an artifact probability threshold (e.g. 0.5, meaning that predictions >= 0.5 will be labeled as artifacts). On inspection, you can deselect marked intervals if you do not agree with the model. Once in the viewer, you can click on "**Highlight model channels** to see which channels were used by the model to make its predictions. "</font> 

<font size="3">Artifacts are best identified by scrolling through the data in segments of 10 seconds while viewing all of the channels. If needed, you can adjust the timeframe from showing the entire plot at once down to merely several milliseconds by clicking and dragging the edges of the slider at the bottom. Scrolling can also be done by dragging-and-dropping with the mouse or trackpad on the channel- or time-axis labels or selecting the 3. button in the taskbar (see below) and then dragging-and-dropping on the plot directly. </font>

<font size="3">The taskbar is located at the top right of the plot. From left to right, the following buttons are integrated:</font>
1. Take a picture of the (annotated) EEG signal and download it as a .png file.
2. Select an area to zoom in on (click-and-drag).
3. Move view along the channel- or time-axis, or both simultaneously (click-and-drag).
4. Select a segment of the data for which to calculate and display the main underlying frequency, as well as a power spectrum in “Power-spectrum” pop-up window, which can be reopened using the “Power spectrum” button in the menu bar (click-and-drag). It is possible to select all channels or only a few desired ones. The latter can be facilitated via the legend explained in Section 3.3.5.
5. (Activated by default) Select a segment of the plot to annotate, which creates a semi-transparent red box spanning the entire vertical axis in view (currently only annotations across all channels are supported) across the selected time interval (click-and-drag). These annotations can be freely adjusted in their position and size, or removed entirely (with button 6.), when clicked on. 
6. Delete the currently selected annotation.
7. Zoom in.
8. Zoom out.
9. Zoom out as much as necessary to show all channels for the entire duration of the recording (or segment), including all peaks in the data (potentially produced by artifacts). 
10. Display a ruler from both axes to the datapoint currently hovered on.

##### Saving changes

<font size="3">After you have finished marking bad channels and annotating artifacts, don't forget to save changes. The save-file will be saved to the "*save_files*" directory as defined earlier when you click on the "**Quit**" button at the top right. Alternatively, use the **"Save to"** button to give the save-file a custom name through the GUI.</font>

In [ ]:
from RV import run_viewer

# sampling frequency to be used for visualizing
downsample_artifact_removal_freq = 200
segment_size = 30

parameters = {'resampling_rate': downsample_artifact_removal_freq, 'segment_size': segment_size}

# Run Robin's viewer
run_viewer(filtered_signal, os.path.join(save_files_folder, 'demo_signal_marked.fif'), parameters)


In [ ]:
save_file_name = "demo_signal_marked.fif"
save_file_path = os.path.join(save_files_folder, save_file_name)
# Load the signal
marked_signal = mne.io.read_raw_fif(save_file_path, preload=True, verbose='INFO')

In [ ]:
# Total number of channels
print("Number of channels: {}".format(len(marked_signal.ch_names)))
# Bad channels marked
print("Bad channels: {}".format(marked_signal.info['bads']))
# Annotations
print(marked_signal.annotations)